In [ ]:
!git clone https://github.com/SKTBrain/KoBERT.git
!cd KoBERTz
!pip install -r requirements.txt
!pip install .
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import re
import os
#import torch
#from torch import nn
#import torch.nn.functional as F
#import torch.optim as optim
#from torch.utils.data import Dataset, DataLoader
#import gluonnlp as nlp
import numpy as np
#from tqdm import tqdm, tqdm_notebook
#from kobert.utils import get_tokenizer
#from kobert.pytorch_kobert import get_pytorch_kobert_model
#from transformers import AdamW
#from transformers.optimization import get_cosine_schedule_with_warmup
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss, accuracy_score,f1_score
import os, sys
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
##GPU 사용 시
#device = torch.device("cuda:0")

In [ ]:
#bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
path = "gdrive/MyDrive/Colab Notebooks/[Dacon]_Climate_Technology_Classification/open/"
sample_submission=pd.read_csv(path+'/sample_submission.csv')

In [ ]:
train=pd.read_csv(path+'train.csv')
test=pd.read_csv(path+'test.csv')

In [ ]:
def clean_text(sent):
  for i in range(len(sent)):
    sent = re.sub("[^가-힣ㄱ-하-ㅣa-zA-Z.]", " ", sent)
  return sent

In [ ]:
train=train[['과제명', '요약문_연구목표', '요약문_연구내용', '요약문_기대효과', '요약문_한글키워드', 'label']]
test=test[['과제명', '요약문_연구목표', '요약문_연구내용', '요약문_기대효과', '요약문_한글키워드']]

train['요약문_연구목표'].fillna('NAN', inplace=True)
train['요약문_연구내용'].fillna('NAN', inplace=True)
train['요약문_기대효과'].fillna('NAN', inplace=True)
train['요약문_한글키워드'].fillna('NAN', inplace=True)
#train['요약문_영문키워드'].fillna('NAN', inplace=True)

test['요약문_연구목표'].fillna('NAN', inplace=True)
test['요약문_연구내용'].fillna('NAN', inplace=True)
test['요약문_기대효과'].fillna('NAN', inplace=True)
test['요약문_한글키워드'].fillna('NAN', inplace=True)
#test['요약문_영문키워드'].fillna('NAN', inplace=True)

In [ ]:
for i in range(len(train)):
  if len(train.요약문_연구목표[i])>200:
    train.요약문_연구목표[i] = train.요약문_연구목표[i][:120] + train.요약문_연구목표[i][-80:]
  if len(train.요약문_연구내용[i])>200:
    train.요약문_연구내용[i] = train.요약문_연구내용[i][:120] + train.요약문_연구내용[i][-80:]
  if len(train.요약문_기대효과[i])>200:
    train.요약문_기대효과[i] = train.요약문_기대효과[i][:120] + train.요약문_기대효과[i][-80:]
  if len(train.요약문_한글키워드[i])>200:
    train.요약문_한글키워드[i] = train.요약문_한글키워드[i][:120] + train.요약문_한글키워드[i][-80:]
  #if len(train.요약문_영문키워드[i])>200:
    #train.요약문_영문키워드[i] = train.요약문_영문키워드[i][:120] + train.요약문_영문키워드[i][-80:]


In [ ]:
train['과제명']  = train.과제명.apply(clean_text)
train['요약문_연구목표']  = train.요약문_연구목표.apply(clean_text)
train['요약문_연구내용']  = train.요약문_연구내용.apply(clean_text)
train['요약문_기대효과']  = train.요약문_기대효과.apply(clean_text)
train['요약문_한글키워드']  = train.요약문_한글키워드.apply(clean_text)
#train['요약문_영문키워드']  = train.요약문_영문키워드.apply(clean_text)
train.head(3)

,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,label
0,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,새로운 해충분류군의 동정기술 개발 및 유입확산 추적,가 외래 및 돌발해충의 발생조사 및 종 동정 대상해충 최근 새...,새로운 돌발 및 외래해충의 신속 정확한 동정법 향상 돌발 및 외래해충의...,뉴클레오티드 염기서열 분자마커 종 동정 침샘 전사체,24
1,대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,최종목표 TRAIL 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구. 발굴된...,차년도 Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자 ...,TRAIL 내성 특이적 표적분자를 발굴하고 이를 이용한 TRAIL 효과 증진...,대장암 항암제 내성 세포사멸 유전자발굴,0
2,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,식물계자원 정련 및 최적 신서란 파이버 기초연구 개발 소비자 및 바이어들...,식물계자원 정련 및 최적 신서란 파이버 기초연구 개발 - Tencel Bam...,국내 독자적인 비목질계 셀룰로오스 자원의 파이버 및 부직포 제조 등의 기술 확보...,기능성 샐룰로오스 파이버 천연섬유 기능성 부직포 뷰티 amp 케어 제품 미용 솜,0


In [ ]:
for i in range(len(test)):
  if len(test.요약문_연구목표[i])>200:
    test.요약문_연구목표[i] = test.요약문_연구목표[i][:120] + test.요약문_연구목표[i][-80:]
  if len(test.요약문_연구내용[i])>200:
    test.요약문_연구내용[i] = test.요약문_연구내용[i][:120] + test.요약문_연구내용[i][-80:]
  if len(test.요약문_기대효과[i])>200:
    test.요약문_기대효과[i] = test.요약문_기대효과[i][:120] + test.요약문_기대효과[i][-80:]
  if len(test.요약문_한글키워드[i])>200:
    test.요약문_한글키워드[i] = test.요약문_한글키워드[i][:120] + test.요약문_한글키워드[i][-80:]
  #if len(test.요약문_영문키워드[i])>200:
    #test.요약문_영문키워드[i] = test.요약문_영문키워드[i][:120] + test.요약문_영문키워드[i][-80:]

In [ ]:
test['과제명']  = test.과제명.apply(clean_text)
test['요약문_연구목표']  = test.요약문_연구목표.apply(clean_text)
test['요약문_연구내용']  = test.요약문_연구내용.apply(clean_text)
test['요약문_기대효과']  = test.요약문_기대효과.apply(clean_text)
test['요약문_한글키워드']  = test.요약문_한글키워드.apply(clean_text)
#test['요약문_영문키워드']  = test.요약문_영문키워드.apply(clean_text)
test.head(3)

,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드
0,R-FSSW 기술 적용 경량 차체 부품 개발 및 품질 평가를 위한 채널 C-SC...,차체 점용접부의 품질 검사를 위한 채널 무선 기반 C-Scan 탐촉자 개발 ...,차년도 . 개발 탐촉 시스템의 성능 평가 위한 표준 시편 제작 시스템...,기술적 파급효과 - 본 연구에서 개발된 R-FSSW 접합 기술은 기존 제조...,마찰교반점용접 비파괴 검사 초음파 탐상 씨 스캔 용접 품질 평가
1,다입자계를 묘사하는 편미분방정식에 대한 연구,자연계에는 입자의 개수가 아주 큰 다양한 다입자계가 존재한다. 이런 다입자계의 효...,연구과제 . 무한입자계의 동역학 작용소 operator 방정식에 대한 연구 ...,본 연구는 물리학에서 중요한 대상인 다입자계를 묘사하는 모델방정식의 정당성을 보장하...,다체계 방정식 동역학의 안정성 양자역학 고전역학 평균장 극한 고전극한 비상대론적 극한
2,저출생체중아 가족을 위한 지역사회중심의 퇴원후 추후관리프로그램 개발,저출생체중아는 정상 체중 출생아에 비하여 운동 정신 행동 및 정서발달장애가 빈번...,본 연구는 퇴원 후 저출생체중아의 퇴원 후 추후관리 프로그램 중 가정방문 모델과 가...,본 연구는 저출생체중아 가족 대상의 지역사회기반의 추후관리 사업에 대한 구체적인 모...,저출생체중아 추후관리 조기개입 자조모임 가정방문


In [ ]:
train.to_csv(path+'clean_train.csv', index=False)
test.to_csv(path+'clean_test.csv', index=False)

In [ ]:
# load preprocessed data to variable
train=pd.read_csv(path+'clean_train.csv')
test=pd.read_csv(path+'clean_test.csv')

In [ ]:
print(train.shape)
print(test.shape)

(174304, 13)
(43576, 12)


In [ ]:
# sampling 90% label-0 data from train data frame in order to solve imbalanced data problem 
sample_df = train[train['label'] != 0]
temp_df = train.query('label==0').sample(frac=0.8) ## 데이터 추출
sample_df = pd.concat([sample_df,temp_df]) ## 데이터 추가    
train = sample_df.reset_index(drop=True) ## 인덱스 초기화

In [ ]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(train, test_size=0.2, random_state=35)
train.head(2)

,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,label
170065,국제 원자력석사과정 운영,o 원전 도입을 계획하는 국가의 원자력 안전 인프라 구축의 일환으로 원자력안전규제 ...,. 교육과정 기획 o . 년 소요의 석사과정을 기본으로 정원은 연간 약 ...,o 원자력안전 선진국으로서 국제사회에 대한 기여를 확대 o 원전 도입을 계획하...,원자력 원자력안전 국제 석사과정,0
146513,건축물 D 프린팅 설계기술 개발 및 테스트베드 추진,목표 소형 건축물 및 비정형 부재 대상 D 프린팅 설계 재료 및 장비 개...,- 세부 건축물 D 프린팅 설계기술 개발 및 테스트베드 수행 연구목표 -...,- 비정형 건축물 시공시간 단축 및 비정형 디자인 시공의 기술적 한계 극복 - ...,D 프린팅 급속시공 건축용 D 프린팅 재료,0


In [ ]:
train.reset_index(drop=True, inplace=True)
train.head(2)

,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,label
0,국제 원자력석사과정 운영,o 원전 도입을 계획하는 국가의 원자력 안전 인프라 구축의 일환으로 원자력안전규제 ...,. 교육과정 기획 o . 년 소요의 석사과정을 기본으로 정원은 연간 약 ...,o 원자력안전 선진국으로서 국제사회에 대한 기여를 확대 o 원전 도입을 계획하...,원자력 원자력안전 국제 석사과정,0
1,건축물 D 프린팅 설계기술 개발 및 테스트베드 추진,목표 소형 건축물 및 비정형 부재 대상 D 프린팅 설계 재료 및 장비 개...,- 세부 건축물 D 프린팅 설계기술 개발 및 테스트베드 수행 연구목표 -...,- 비정형 건축물 시공시간 단축 및 비정형 디자인 시공의 기술적 한계 극복 - ...,D 프린팅 급속시공 건축용 D 프린팅 재료,0


In [ ]:
validation.reset_index(drop=True, inplace=True)
validation.head(2)

,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,label
0,오픈소스 코드 분석 기반 코딩 학습자 평가용 인공지능 역량진단 및 추천 시스템 개발,- 코딩평가 오픈소스 및 작성코드 분석을 통한 학습자 평가 인공지능 모델 개발 ...,코딩평가 오픈소스 및 작성코드 분석을 통한 학습자 평가 인공지능 모델 개발 ...,본 과제 개발을 통해 대학교 등에서 사용하고 있는 온라인 웹기반 코딩 교육시스템을 ...,코딩 교육 학습관리시스템 인공지능 추천시스템 머신러닝,0
1,토양 내 가 크롬 유도 독성 메커니즘 구명에 의한 식물 크롬 독성 저해 기술 개발,차 년도 토양에 망간 처리 농도를 변화시키면서 망간에 의한 식물 독성 평가 ...,차 년도 - 망간과 크롬으로 오염되지 않은 일반 토양을 채취하여 토양의 기본 ...,- 망간 함량이 높은 토양 조건에서 가 크롬과 가 크롬의 독성평가를 통해 토양오...,가 크롬 가 크롬 식물 독성 토양 망간 철,0


In [ ]:
train.to_csv(path+'clean_train_split_v2.0.csv', index=False)
validation.to_csv(path+'clean_validation_split_v2.0.csv', index=False)